# Recommender Systems 2020/21

### Practice - Hyperparameter optimization with Skopt

### Hyperparameter optimization is essential to achieve the best recommendation quality

### Hyperparameter optimization strategies
* Grid-search (use predefined list of values): simple but often unpractically slow and very difficult to fine tune
* Random-search (use predefined range and distribution): very parallelizable and effective
* Bayesian-search (random search + gaussian process): less parallelizable, good exploration-exploitation tradeoff


### In the course repo you will find a BayesianSearch object in the ParameterTuning folder. That is a simple wrapper of another library and its purpose is to provide a very simple way to tune some of the most common parameters. 

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()
ICM_all = data_loaded.get_ICM_from_name("ICM_genres")

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10217, feature occurrences: 108563, density 9.95E-04
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10237, feature occurrences: 130127, density 1.19E-03




### How do we perform hyperparameter optimization?
* Split the data in three *disjoint* sets: training, validation and testing data
* Define a set of hyperparameters with the range and distribution
* Explore hyperparameter space and select those with the best recommendation quality on the *validation* data (including the number of epochs for ML algorithms)
* Given the best hyperparameters, fit the model again using the union of training and validation data.
* Evaluate this last model on the testing data.

### Step 1: Split the data and create the evaluator objects

In [2]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

### Step 2: Define hyperparameter set for the desired model, in this case ItemBasedKNN

In [3]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
hyperparameters_range_dictionary["similarity"] = Categorical(["cosine"])
hyperparameters_range_dictionary["normalize"] = Categorical([True, False])


### Step 3: Create SearchBayesianSkopt object, providing the desired recommender class and evaluator objects

In [4]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNCFRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

### Step 4: Provide the data needed to create an instance of the model, one trained only on URM_train, the other on URM_train_validation

In [5]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [6]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

### Step 5: Create a result folder and select the number of cases (50 with 30% random is a good number)

In [7]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   

### Step 5: Run!

In [8]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
ItemKNNCFRecommender: URM Detected 73 (0.68 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 789, 'shrink': 259, 'similarity': 'cosine', 'normalize': True}
Similarity column 10681 ( 100 % ), 987.88 column/sec, elapsed time 0.18 min
EvaluatorHoldout: Processed 32000 ( 45.95% ) in 30.93 sec. Users per second: 1034
EvaluatorHoldout: Processed 63000 ( 90.47% ) in 1.03 min. Users per second: 1022
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 1.13 min. Users per second: 1023
SearchBayesianSkopt: New best config found. Config 0: {'topK': 789, 'shrink': 259, 'similarity': 'cosine', 'normalize': True} - results: ROC_AUC: 0.4390522, PRECISION: 0.1906800, PRECISION_RECALL_MIN_DEN: 0.2388892, RECALL: 0.1559166, MAP: 0.1239337, MRR: 0.4154496, NDCG: 0.1649812, F1: 0.1715549, HIT_RATE: 1.9067997, ARHR: 0.6396330, NOVELTY: 0.0086132, AVERAGE_POPULARITY: 0.5370173, DIVERSITY_MEAN_INTER_LIST: 0.8917573, DIVERSITY_HERFINDAHL: 0

Iteration No: 5 ended. Search finished for the next optimal point.
Time taken: 108.1322
Function value obtained: -0.1225
Current minimum: -0.1245
Iteration No: 6 started. Searching for the next optimal point.
ItemKNNCFRecommender: URM Detected 73 (0.68 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 1000, 'shrink': 0, 'similarity': 'cosine', 'normalize': True}
Similarity column 10681 ( 100 % ), 947.15 column/sec, elapsed time 0.19 min
EvaluatorHoldout: Processed 30000 ( 43.08% ) in 30.12 sec. Users per second: 996
EvaluatorHoldout: Processed 60000 ( 86.16% ) in 1.00 min. Users per second: 995
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 1.17 min. Users per second: 991
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'topK': 1000, 'shrink': 0, 'similarity': 'cosine', 'normalize': True} - results: ROC_AUC: 0.4372607, PRECISION: 0.1892051, PRECISION_RECALL_MIN_DEN: 0.2373137, RECALL: 0.1550236, MAP: 0.1228231, MRR: 0.4126555, NDCG: 0.1637222, F1: 0.1704172, HIT_RATE:

ItemKNNCFRecommender: Saving complete


In [9]:
from Base.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata

{'algorithm_name_search': 'SearchBayesianSkopt',
 'algorithm_name_recommender': 'ItemKNNCFRecommender',
 'exception_list': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'hyperparameters_list': [{'topK': 789,
   'shrink': 259,
   'similarity': 'cosine',
   'normalize': True},
  {'topK': 670, 'shrink': 121, 'similarity': 'cosine', 'normalize': True},
  {'topK': 80, 'shrink': 487, 'similarity': 'cosine', 'normalize': False},
  {'topK': 5, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True},
  {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True},
  {'topK': 1000, 'shrink': 0, 'similarity': 'cosine', 'normalize': True},
  {'topK': 1000, 'shrink': 0, 'similarity': 'cosine', 'normalize': False},
  {'topK': 710, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True},
  {'topK': 734, 'shrink': 0, 'similarity': 'cosine', 'normalize': True},
  {'topK': 769, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True}],
 'hyperparame

In [10]:
hyperparameters_list = search_metadata["hyperparameters_list"]
hyperparameters_list

[{'topK': 789, 'shrink': 259, 'similarity': 'cosine', 'normalize': True},
 {'topK': 670, 'shrink': 121, 'similarity': 'cosine', 'normalize': True},
 {'topK': 80, 'shrink': 487, 'similarity': 'cosine', 'normalize': False},
 {'topK': 5, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True},
 {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True},
 {'topK': 1000, 'shrink': 0, 'similarity': 'cosine', 'normalize': True},
 {'topK': 1000, 'shrink': 0, 'similarity': 'cosine', 'normalize': False},
 {'topK': 710, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True},
 {'topK': 734, 'shrink': 0, 'similarity': 'cosine', 'normalize': True},
 {'topK': 769, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True}]

In [11]:
result_best_on_test = search_metadata["result_on_last"]
result_best_on_test["10"]

{'ROC_AUC': 0.5334465928491546,
 'PRECISION': 0.305380735391055,
 'PRECISION_RECALL_MIN_DEN': 0.34610293873470677,
 'RECALL': 0.18482762605807215,
 'MAP': 0.2432645075760898,
 'MRR': 0.5859145692113457,
 'NDCG': 0.22233536528638775,
 'F1': 0.2302808389450718,
 'HIT_RATE': 3.0538073539105266,
 'ARHR': 1.0804502676889975,
 'NOVELTY': 0.008668107818871092,
 'AVERAGE_POPULARITY': 0.5181971273808239,
 'DIVERSITY_MEAN_INTER_LIST': 0.9076256000234381,
 'DIVERSITY_HERFINDAHL': 0.9907612594182724,
 'COVERAGE_ITEM': 0.1829416721280779,
 'COVERAGE_ITEM_CORRECT': 0.11328527291452112,
 'COVERAGE_USER': 0.9986834196742895,
 'COVERAGE_USER_CORRECT': 0.827213142906208,
 'DIVERSITY_GINI': 0.013167507472345191,
 'SHANNON_ENTROPY': 7.468182907974959}

In [12]:
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

{'topK': 670, 'shrink': 121, 'similarity': 'cosine', 'normalize': True}

## An example with earlystopping, for BPRMF

In [13]:
hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["sgd_mode"] = Categorical(["sgd", "adagrad", "adam"])
hyperparameters_range_dictionary["epochs"] = Categorical([1500])
hyperparameters_range_dictionary["num_factors"] = Integer(1, 200)
hyperparameters_range_dictionary["batch_size"] = Categorical([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
hyperparameters_range_dictionary["positive_reg"] = Real(low = 1e-5, high = 1e-2, prior = 'log-uniform')
hyperparameters_range_dictionary["negative_reg"] = Real(low = 1e-5, high = 1e-2, prior = 'log-uniform')
hyperparameters_range_dictionary["learning_rate"] = Real(low = 1e-4, high = 1e-1, prior = 'log-uniform')


In [14]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [15]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)

In [16]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)

In [17]:
n_cases = 5
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   

In [18]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython

recommender_class = MatrixFactorization_BPR_Cython

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [19]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

Iteration No: 1 started. Evaluating function at random point.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 73 (0.68 %) cold items.
SearchBayesianSkopt: Testing config: {'sgd_mode': 'adam', 'epochs': 1500, 'num_factors': 9, 'batch_size': 256, 'positive_reg': 0.0009557358739689882, 'negative_reg': 0.0013254069621555342, 'learning_rate': 0.025468411698118695}
MF_BPR: Processed 69632 ( 99.63% ) in 0.87 seconds. BPR loss 8.12E-03. Sample per second: 79753
MF_BPR: Epoch 1 of 1500. Elapsed time 0.28 sec
MF_BPR: Processed 69632 ( 99.63% ) in 1.14 seconds. BPR loss 7.97E-02. Sample per second: 61091
MF_BPR: Epoch 2 of 1500. Elapsed time 0.54 sec
MF_BPR: Processed 69632 ( 99.63% ) in 0.41 seconds. BPR loss 1.13E+00. Sample per second: 167902
MF_BPR: Epoch 3 of 1500. Elapsed time 0.82 sec
MF_BPR: Processed 69632 ( 99.63% ) in 0.69 seconds. BPR loss 6.03E+00. Sample per second: 101233
MF_BPR: Epoch 4 of 1500. Elapsed time 1.09 sec
MF_BPR: Processed 69632 ( 99.63% ) in 0.97 seconds. BPR

MF_BPR: Processed 69632 ( 99.63% ) in 1.21 seconds. BPR loss 9.92E+01. Sample per second: 57582
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 52000 ( 74.68% ) in 30.22 sec. Users per second: 1721
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 40.17 sec. Users per second: 1734
MF_BPR: CUTOFF: 10 - ROC_AUC: 0.1537910, PRECISION: 0.0504086, PRECISION_RECALL_MIN_DEN: 0.0639268, RECALL: 0.0418918, MAP: 0.0275932, MRR: 0.1224930, NDCG: 0.0394437, F1: 0.0457572, HIT_RATE: 0.5040856, ARHR: 0.1637317, NOVELTY: 0.0119237, AVERAGE_POPULARITY: 0.2671998, DIVERSITY_MEAN_INTER_LIST: 0.9582472, DIVERSITY_HERFINDAHL: 0.9958233, COVERAGE_ITEM: 0.3268421, COVERAGE_ITEM_CORRECT: 0.0552383, COVERAGE_USER: 0.9965225, COVERAGE_USER_CORRECT: 0.2803171, DIVERSITY_GINI: 0.0609791, SHANNON_ENTROPY: 9.4078383, 

MF_BPR: Convergence reached! Terminating at epoch 30. Best value for 'MAP' at epoch 5 is 0.0290. Elapsed time 4.10 min
MF_BPR: Epoch 30 of 1500. Elapsed time 4.10 min
EvaluatorHoldout: Proce

MF_BPR: Processed 69824 ( 99.91% ) in 4.26 seconds. BPR loss 3.98E-02. Sample per second: 16400
MF_BPR: Epoch 19 of 1500. Elapsed time 3.35 min
MF_BPR: Processed 69824 ( 99.91% ) in 4.28 seconds. BPR loss 3.98E-02. Sample per second: 16310
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 49000 ( 70.37% ) in 30.29 sec. Users per second: 1618
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 43.77 sec. Users per second: 1591
MF_BPR: CUTOFF: 10 - ROC_AUC: 0.0102222, PRECISION: 0.0021469, PRECISION_RECALL_MIN_DEN: 0.0023346, RECALL: 0.0009105, MAP: 0.0006710, MRR: 0.0059604, NDCG: 0.0008586, F1: 0.0012787, HIT_RATE: 0.0214691, ARHR: 0.0060615, NOVELTY: 0.0149522, AVERAGE_POPULARITY: 0.0248332, DIVERSITY_MEAN_INTER_LIST: 0.9987953, DIVERSITY_HERFINDAHL: 0.9998781, COVERAGE_ITEM: 1.0000000, COVERAGE_ITEM_CORRECT: 0.0924071, COVERAGE_USER: 0.9965225, COVERAGE_USER_CORRECT: 0.0207648, DIVERSITY_GINI: 0.7029924, SHANNON_ENTROPY: 13.1780043, 

MF_BPR: Epoch 20 of 1500. Elapsed time 4.15 m

MF_BPR: Processed 69872 ( 99.98% ) in 0.88 seconds. BPR loss 1.81E-04. Sample per second: 79744
MF_BPR: Epoch 11 of 1500. Elapsed time 1.36 min
MF_BPR: Processed 69872 ( 99.98% ) in 0.92 seconds. BPR loss 1.80E-04. Sample per second: 76100
MF_BPR: Epoch 12 of 1500. Elapsed time 1.36 min
MF_BPR: Processed 69872 ( 99.98% ) in 0.96 seconds. BPR loss 1.76E-04. Sample per second: 72623
MF_BPR: Epoch 13 of 1500. Elapsed time 1.36 min
MF_BPR: Processed 69872 ( 99.98% ) in 1.00 seconds. BPR loss 1.77E-04. Sample per second: 69658
MF_BPR: Epoch 14 of 1500. Elapsed time 1.36 min
MF_BPR: Processed 69872 ( 99.98% ) in 0.05 seconds. BPR loss 1.79E-04. Sample per second: 1424717
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 53000 ( 76.11% ) in 30.39 sec. Users per second: 1744
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 41.15 sec. Users per second: 1692
MF_BPR: CUTOFF: 10 - ROC_AUC: 0.0026009, PRECISION: 0.0005601, PRECISION_RECALL_MIN_DEN: 0.0005708, RECALL: 0.0001136, MAP: 0.000131

MF_BPR: Processed 69632 ( 99.63% ) in 4.38 seconds. BPR loss 3.94E-02. Sample per second: 15882
MF_BPR: Epoch 6 of 1500. Elapsed time 1.13 min
MF_BPR: Processed 69632 ( 99.63% ) in 4.78 seconds. BPR loss 3.97E-02. Sample per second: 14573
MF_BPR: Epoch 7 of 1500. Elapsed time 1.21 min
MF_BPR: Processed 69632 ( 99.63% ) in 5.15 seconds. BPR loss 4.01E-02. Sample per second: 13512
MF_BPR: Epoch 8 of 1500. Elapsed time 1.28 min
MF_BPR: Processed 69632 ( 99.63% ) in 4.60 seconds. BPR loss 4.02E-02. Sample per second: 15150
MF_BPR: Epoch 9 of 1500. Elapsed time 1.35 min
MF_BPR: Processed 69632 ( 99.63% ) in 5.07 seconds. BPR loss 4.00E-02. Sample per second: 13728
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 49000 ( 70.37% ) in 30.27 sec. Users per second: 1619
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 42.87 sec. Users per second: 1624
MF_BPR: CUTOFF: 10 - ROC_AUC: 0.0114197, PRECISION: 0.0023221, PRECISION_RECALL_MIN_DEN: 0.0025072, RECALL: 0.0009449, MAP: 0.0007697, MRR

...

MF_BPR: Epoch 611 of 840. Elapsed time 42.93 min
MF_BPR: Processed 69878 ( 100.00% ) in 5.04 seconds. BPR loss 1.34E+01. Sample per second: 13857
MF_BPR: Epoch 612 of 840. Elapsed time 43.00 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.35 seconds. BPR loss 1.34E+01. Sample per second: 16062
MF_BPR: Epoch 613 of 840. Elapsed time 43.08 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.57 seconds. BPR loss 1.34E+01. Sample per second: 15298
MF_BPR: Epoch 614 of 840. Elapsed time 43.15 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.79 seconds. BPR loss 1.34E+01. Sample per second: 14577
MF_BPR: Epoch 615 of 840. Elapsed time 43.22 min
MF_BPR: Processed 69878 ( 100.00% ) in 5.03 seconds. BPR loss 1.35E+01. Sample per second: 13889
MF_BPR: Epoch 616 of 840. Elapsed time 43.29 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.27 seconds. BPR loss 1.33E+01. Sample per second: 16367
MF_BPR: Epoch 617 of 840. Elapsed time 43.36 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.60 seconds. BPR loss 1.35E+01. Sam

MF_BPR: Processed 69878 ( 100.00% ) in 4.64 seconds. BPR loss 1.42E+01. Sample per second: 15056
MF_BPR: Epoch 668 of 840. Elapsed time 46.93 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.78 seconds. BPR loss 1.42E+01. Sample per second: 14613
MF_BPR: Epoch 669 of 840. Elapsed time 47.00 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.94 seconds. BPR loss 1.42E+01. Sample per second: 14156
MF_BPR: Epoch 670 of 840. Elapsed time 47.07 min
MF_BPR: Processed 69878 ( 100.00% ) in 5.08 seconds. BPR loss 1.43E+01. Sample per second: 13769
MF_BPR: Epoch 671 of 840. Elapsed time 47.14 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.21 seconds. BPR loss 1.43E+01. Sample per second: 16589
MF_BPR: Epoch 672 of 840. Elapsed time 47.21 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.35 seconds. BPR loss 1.43E+01. Sample per second: 16081
MF_BPR: Epoch 673 of 840. Elapsed time 47.28 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.48 seconds. BPR loss 1.43E+01. Sample per second: 15595
MF_BPR: Epoch 674 of 840. E

MF_BPR: Epoch 724 of 840. Elapsed time 50.82 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.16 seconds. BPR loss 1.52E+01. Sample per second: 16813
MF_BPR: Epoch 725 of 840. Elapsed time 50.89 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.30 seconds. BPR loss 1.51E+01. Sample per second: 16239
MF_BPR: Epoch 726 of 840. Elapsed time 50.96 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.46 seconds. BPR loss 1.51E+01. Sample per second: 15684
MF_BPR: Epoch 727 of 840. Elapsed time 51.03 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.65 seconds. BPR loss 1.53E+01. Sample per second: 15022
MF_BPR: Epoch 728 of 840. Elapsed time 51.10 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.82 seconds. BPR loss 1.50E+01. Sample per second: 14496
MF_BPR: Epoch 729 of 840. Elapsed time 51.17 min
MF_BPR: Processed 69878 ( 100.00% ) in 5.01 seconds. BPR loss 1.52E+01. Sample per second: 13938
MF_BPR: Epoch 730 of 840. Elapsed time 51.24 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.24 seconds. BPR loss 1.53E+01. Sam

MF_BPR: Processed 69878 ( 100.00% ) in 5.08 seconds. BPR loss 1.61E+01. Sample per second: 13768
MF_BPR: Epoch 781 of 840. Elapsed time 54.84 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.25 seconds. BPR loss 1.60E+01. Sample per second: 16460
MF_BPR: Epoch 782 of 840. Elapsed time 54.91 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.40 seconds. BPR loss 1.59E+01. Sample per second: 15876
MF_BPR: Epoch 783 of 840. Elapsed time 54.98 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.55 seconds. BPR loss 1.60E+01. Sample per second: 15370
MF_BPR: Epoch 784 of 840. Elapsed time 55.05 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.68 seconds. BPR loss 1.60E+01. Sample per second: 14935
MF_BPR: Epoch 785 of 840. Elapsed time 55.11 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.82 seconds. BPR loss 1.60E+01. Sample per second: 14510
MF_BPR: Epoch 786 of 840. Elapsed time 55.18 min
MF_BPR: Processed 69878 ( 100.00% ) in 5.00 seconds. BPR loss 1.61E+01. Sample per second: 13974
MF_BPR: Epoch 787 of 840. E

MF_BPR: Epoch 837 of 840. Elapsed time 58.78 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.68 seconds. BPR loss 1.66E+01. Sample per second: 14935
MF_BPR: Epoch 838 of 840. Elapsed time 58.85 min
MF_BPR: Processed 69878 ( 100.00% ) in 4.91 seconds. BPR loss 1.69E+01. Sample per second: 14237
MF_BPR: Epoch 839 of 840. Elapsed time 58.92 min
MF_BPR: Processed 69878 ( 100.00% ) in 5.14 seconds. BPR loss 1.68E+01. Sample per second: 13601
MF_BPR: Epoch 840 of 840. Elapsed time 58.99 min
MF_BPR: Terminating at epoch 840. Elapsed time 58.99 min
EvaluatorHoldout: Processed 52000 ( 74.51% ) in 30.06 sec. Users per second: 1730
EvaluatorHoldout: Processed 69786 ( 100.00% ) in 39.86 sec. Users per second: 1751
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'sgd_mode': 'adam', 'epochs': 840, 'num_factors': 200, 'batch_size': 1, 'positive_reg': 2.1264549316860416e-05, 'negative_reg': 0.00033305173444924584, 'learning_rate': 0.0001015122

In [20]:
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_list = search_metadata["hyperparameters_list"]
hyperparameters_list

[{'sgd_mode': 'adam',
  'epochs': 1500,
  'num_factors': 9,
  'batch_size': 256,
  'positive_reg': 0.0009557358739689882,
  'negative_reg': 0.0013254069621555342,
  'learning_rate': 0.025468411698118695},
 {'sgd_mode': 'adagrad',
  'epochs': 1500,
  'num_factors': 200,
  'batch_size': 64,
  'positive_reg': 1e-05,
  'negative_reg': 1e-05,
  'learning_rate': 0.0001},
 {'sgd_mode': 'sgd',
  'epochs': 1500,
  'num_factors': 1,
  'batch_size': 16,
  'positive_reg': 0.01,
  'negative_reg': 0.01,
  'learning_rate': 0.1},
 {'sgd_mode': 'adam',
  'epochs': 1500,
  'num_factors': 199,
  'batch_size': 256,
  'positive_reg': 1e-05,
  'negative_reg': 1e-05,
  'learning_rate': 0.0001},
 {'sgd_mode': 'adam',
  'epochs': 1500,
  'num_factors': 200,
  'batch_size': 1,
  'positive_reg': 2.1264549316860416e-05,
  'negative_reg': 0.00033305173444924584,
  'learning_rate': 0.00010151223842175566}]

In [21]:
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

{'sgd_mode': 'adam',
 'epochs': 840,
 'num_factors': 200,
 'batch_size': 1,
 'positive_reg': 2.1264549316860416e-05,
 'negative_reg': 0.00033305173444924584,
 'learning_rate': 0.00010151223842175566}

### How to use the predefined hyperparameter range

#### Function runParameterSearch_Collaborative takes as input all needed to optimize any of the recommenders in the framework as well as other parameters such as which similarity heuristics to use for the KNNs, whether to parallelize the training of KNNs and so on...

In [22]:
similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"]

In [23]:
from ParameterTuning.run_parameter_search import runParameterSearch_Collaborative, runParameterSearch_Content
from Base.NonPersonalizedRecommender import TopPop, Random
from GraphBased.P3alphaRecommender import P3alphaRecommender

recommender_class = TopPop

runParameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_validation,
       metric_to_optimize = metric_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

SearchSingleCase: Resuming 'TopPopRecommender' Failed, no such file exists.
TopPopRecommender: URM Detected 73 (0.68 %) cold items.
SearchSingleCase: Testing config: {}
EvaluatorHoldout: Processed 66000 ( 94.78% ) in 30.25 sec. Users per second: 2182
EvaluatorHoldout: Processed 69635 ( 100.00% ) in 31.89 sec. Users per second: 2183
SearchSingleCase: New best config found. Config 0: {} - results: ROC_AUC: 0.3147372, PRECISION: 0.1120011, PRECISION_RECALL_MIN_DEN: 0.1336714, RECALL: 0.0782386, MAP: 0.0636972, MRR: 0.2674676, NDCG: 0.0895075, F1: 0.0921239, HIT_RATE: 1.1200115, ARHR: 0.3775154, NOVELTY: 0.0078829, AVERAGE_POPULARITY: 0.8419826, DIVERSITY_MEAN_INTER_LIST: 0.3943088, DIVERSITY_HERFINDAHL: 0.9394303, COVERAGE_ITEM: 0.0044940, COVERAGE_ITEM_CORRECT: 0.0042131, COVERAGE_USER: 0.9965225, COVERAGE_USER_CORRECT: 0.5519047, DIVERSITY_GINI: 0.0015082, SHANNON_ENTROPY: 4.2380186, 

SearchSingleCase: Saving model in result_experiments/TopPopRecommender

TopPopRecommender: Saving mode

### The process is similar for content-based algorithms.
* Add the ICM data and its name (it will be used for file names)
* Remove the evaluator for earlystopping

In [24]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

recommender_class = ItemKNNCBFRecommender

runParameterSearch_Content(recommender_class,
       URM_train = URM_train,
       ICM_object = ICM_all,
       ICM_name = "ICM_genres",
       URM_train_last_test = URM_train_validation,
       metric_to_optimize = metric_to_optimize,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

### You can use the "partial" function to pre-set all attributes that are needed and use it to loop on the recommender_classes

In [25]:
from Base.NonPersonalizedRecommender import TopPop, Random
from GraphBased.P3alphaRecommender import P3alphaRecommender
import os, multiprocessing
from functools import partial


runParameterSearch_Collaborative_partial = partial(runParameterSearch_Collaborative,
       URM_train = URM_train,
       URM_train_last_test = URM_train_validation,
       metric_to_optimize = metric_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)


In [26]:
collaborative_algorithm_list = [
    Random,
    TopPop,
    P3alphaRecommender,
]

### You can call it as part of a parallel pool, hence parallizing the optimization of the various models

In [27]:
pool = multiprocessing.Pool(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
pool.map(runParameterSearch_Collaborative_partial, collaborative_algorithm_list)

[None, None, None]

### Or sequentially, note that since resume_from_saved = True this new call to the function will just load the optimized values and not re-run it 

In [28]:
for recommender_class in collaborative_algorithm_list:

    try:

        runParameterSearch_Collaborative_partial(recommender_class)

    except Exception as e:

        print("On recommender {} Exception {}".format(recommender_class, str(e)))
        traceback.print_exc()

SearchSingleCase: Resuming 'RandomRecommender'... Loaded 1 configurations.
RandomRecommender: URM Detected 37 (0.35 %) cold items.
SearchSingleCase: Resuming 'RandomRecommender'... Result on last already available.
SearchSingleCase: Resuming 'TopPopRecommender'... Loaded 1 configurations.
TopPopRecommender: URM Detected 37 (0.35 %) cold items.
SearchSingleCase: Resuming 'TopPopRecommender'... Result on last already available.
SearchBayesianSkopt: Resuming 'P3alphaRecommender'... Loaded 5 configurations.
Iteration No: 1 started. Searching for the next optimal point.
Iteration No: 1 ended. Search finished for the next optimal point.
Time taken: 0.2288
Function value obtained: -0.1047
Current minimum: -0.1129
P3alphaRecommender: URM Detected 37 (0.35 %) cold items.
SearchBayesianSkopt: Resuming 'P3alphaRecommender'... Result on last already available.


## How to load and save a model
### The models are saved in two files:
* RECOMMENDER_NAME + "_best_model.zip": contains the model trained on the URM_train
* RECOMMENDER_NAME + "_best_model_last.zip": contains the model trained on the URM_train_validation

#### You can load them using the "load_model" function of the Recommender class, to do so:
* Create an instance of the recommender you want to load and provide the needed URM data and ICM data, if needed. The URM you pass is the one that will be used as user profiles to generate the recommendations.
* Given the instance, call the load_model function
* You can now use the recommender to generate recommendations.

#### If you want to save your own models use the "save_model" function, it takes thesame parameters as "load_model"

In [13]:
from GraphBased.P3alphaRecommender import P3alphaRecommender

recommender_object = P3alphaRecommender(URM_all)
recommender_object

P3alphaRecommender: URM Detected 4 (0.04 %) cold items.


In [14]:
recommender_object.load_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip" )

P3alphaRecommender: Loading model from file 'result_experiments/P3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete


In [15]:
recommender_object.W_sparse

<10681x10681 sparse matrix of type '<class 'numpy.float32'>'
	with 2666401 stored elements in Compressed Sparse Row format>

In [16]:
user_id = 10
recommender_object.recommend(user_id, cutoff = 10)

[19, 22, 7, 175, 178, 139, 605, 404, 133, 1293]

In [17]:
recommender_object.save_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip" )

P3alphaRecommender: Saving model in file 'result_experiments/P3alphaRecommender_my_own_save.zip'
P3alphaRecommender: Saving complete
